# Charity Scraper
### The Plan:
1. Scrape Each Charity's register_index URL from the main search page
1. Open each charity and scrape characteristics
1. Use generated charity_index to filter for websites with desired characteristics
  * income
  * reporting
  * cause helped
  * has website and charity_url

In [55]:
# Initialisation and loadup
from http import server
import pandas as pd
import os
import time

import selenium as sl
from selenium import webdriver
#driverpath = "C:\Program Files\chromedriver.exe"

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC


In [120]:
# initialise the pandas dataframe
columns = ["Index_Link", "Legal_Name", "Town/Suburb","State", "Status", "Size","ABN","Website","Revenue","Cause/s"]
df=pd.DataFrame(columns=columns)
df

,Index_Link,Legal_Name,Town/Suburb,State,Status,Size,ABN,Website,Revenue,Cause/s


In [107]:
# initialise driver
# Assemble the pit crew and start driving

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
pfurl = "https://www.acnc.gov.au/charity/charities?items_per_page=100&f[]=size%3ASmall" #pre filtered url for small businesses and to show 100 results - Page 1
driver.get(pfurl)
driver.implicitly_wait(10)

In [95]:

df.shape[1]

9

In [116]:
def extract_data():
    driver.implicitly_wait(10)
    table=driver.find_element(by=By.TAG_NAME, value="tbody")
    rows=table.find_elements(by=By.TAG_NAME,value="tr")

    i=df.shape[0]
    for row in rows:
        link = row.find_element(by=By.TAG_NAME,value="a").get_attribute("href")
        name =      row.find_elements(by=By.TAG_NAME, value="td")[0].text
        town =      row.find_elements(by=By.TAG_NAME, value="td")[1].text
        state =     row.find_elements(by=By.TAG_NAME, value="td")[2].text
        status =    row.find_elements(by=By.TAG_NAME, value="td")[3].text
        size =      row.find_elements(by=By.TAG_NAME, value="td")[4].text
        abn =       row.find_elements(by=By.TAG_NAME, value="td")[5].text
        df.loc[i]=[link,name,town,state,status,size,abn,None,None,None]
        i+=1
    

In [121]:
df.head

<bound method NDFrame.head of Empty DataFrame
Columns: [Index_Link, Legal_Name, Town/Suburb, State, Status, Size, ABN, Website, Revenue, Cause/s]
Index: []>

In [126]:
#navigate to next page building block
def nav_page():
    totalpages=353
    page=1
    while page < totalpages:
        extract_data()
        driver.find_element(by=By.CLASS_NAME,value="next").find_element(by=By.TAG_NAME,value="button").click()
        page += 1
        time.sleep(20)

driver.get(pfurl)
time.sleep(20)
nav_page()

df.to_csv("/list1.csv")

KeyboardInterrupt: 

In [12]:
# kill driver
driver.quit()